### Loading the libraries

In [137]:
import pandas as pd
import pyodbc
import warnings
warnings.simplefilter("ignore", UserWarning)

import os
from dotenv import load_dotenv
load_dotenv()

from sqlalchemy import create_engine, text
import sqlalchemy


### Setting up the Database connection with MySQL

In [140]:
os.getenv("user")

'root'

In [146]:
conn = pyodbc.connect(
    "DRIVER={MySQL ODBC 8.0 Unicode Driver};"
    f"SERVER={os.getenv('server')};"
    f"PORT={os.getenv('port')};"
    f"DATABASE={os.getenv('database')};"
    f"USER={os.getenv('user')};"
    f"PASSWORD={os.getenv('password')};"
    "OPTION=3;"
)

#conn.close()

#### Testing the DB connection

In [133]:
basic_query = "SELECT * FROM retail_sales"
pd.read_sql(basic_query, conn)

,transactions_id,sale_date,sale_time,customer_id,gender,age,category,quantity,price_per_unit,cogs,total_sale
0,1,2022-12-16,19:10:00,50,Male,34.0,Beauty,3.0,50.0,16.0,150.0
1,2,2022-06-24,10:07:00,104,Female,26.0,Clothing,2.0,500.0,135.0,1000.0
2,3,2022-06-14,07:08:00,114,Male,50.0,Electronics,1.0,30.0,8.1,30.0
3,4,2023-08-27,18:12:00,3,Male,37.0,Clothing,1.0,500.0,200.0,500.0
4,5,2023-09-05,22:10:00,3,Male,30.0,Beauty,2.0,50.0,24.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,2023-12-07,18:49:00,2,Male,62.0,Clothing,1.0,50.0,25.0,50.0
1996,1997,2023-10-29,21:09:00,99,Male,52.0,Beauty,3.0,30.0,10.2,90.0
1997,1998,2023-10-20,19:53:00,90,Female,23.0,Beauty,4.0,25.0,24.0,100.0
1998,1999,2022-11-12,13:06:00,61,Female,36.0,Electronics,3.0,50.0,13.0,150.0


In [105]:
len(basic)

2000

### Trying SQLAlchemy

In [129]:
# Define MySQL Connection String
db_user = "root"
db_password = "Mymummy1%40"
db_host = "localhost"  # Change to your MySQL server if needed
db_port = "3306"  # Default MySQL port
db_name = "sql_project_1"  # Change this to your desired database name

# Create SQLAlchemy Engine
engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Test connection
with engine.connect() as conn:
    print("✅ Connected to MySQL successfully!")

✅ Connected to MySQL successfully!


In [125]:
conn = pymysql.connect(db='sql_project_1', user='root', passwd='Mymummy1@', host='localhost', port=3306)
conn.close()

In [127]:
f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

'mysql+pymysql://root:Mymummy1@@localhost:3306/sql_project_1'